In [90]:
import pickle
ss = {"sp" : [1, 2, 3], "f0" : [4,5,6] }
dd = { "v" : [ss]*5}
# d = { "val" : dd, "key" : ss }
print(dd['v'][0])
# afile = open(r'd.pkl', 'wb')

# pickle.dump(d, afile)
# afile.close()

{'sp': [1, 2, 3], 'f0': [4, 5, 6]}


In [26]:

foo = pickle.load(open("./train_data/d.pkl", "rb"))
idx = np.where(foo[1]["note"] == 31.)
print(foo[1]["sp"][idx])


[[6.53763706e-01 6.58781104e-01 6.69004692e-01 ... 2.09629691e-10
  2.15898147e-10 2.17976607e-10]]


In [2]:
import os
import json
import pickle
import soundfile as sf
import pyworld as pw
import matplotlib.pyplot as plt
import numpy as np

aiueo = ['a','i','u','e','o']

all_data = []
note_step=0.25
frame_period = 10.0 # millisecond
wav_dir = 'gliss_dataset'
def get_f0_index_tbl(f0,min_note,max_note,note_step=0.25):
    f0_note = 12*np.log2(f0/440)+49
    out_len = int((max_note-min_note)/note_step+1)
    note_lin =np.linspace(min_note, max_note, num=out_len)
    idx_out = np.zeros(out_len,dtype = int)
    for i in range(out_len):
        idx_out[i] = np.argmin(abs(f0_note-note_lin[i]))
    return idx_out
    
for who in os.listdir(wav_dir):

    rangePath = os.path.join(wav_dir,who,"info.json")
    with open(rangePath, 'r') as f:
        info = json.load(f)
        [min_note,max_note] = info['pRange']

    print(min_note,max_note)
    key_label_dic = {}
    for v in aiueo:
        path = os.path.join(wav_dir,who,v)
        print(path)
        x,fs = sf.read(path+'.wav')
        x = (x[:,0] + x[:,1])/2 
        x = x/max(x)*0.5
        f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)
        f0 = f0[1:-1]
        sp = sp[1:-1,:]
        ap = ap[1:-1,:]
        file_cnt = 2
        while os.path.isfile(path+str(file_cnt)+'.wav'):
            print(path+str(file_cnt)+'.wav')
            x2,_ = sf.read(path+str(file_cnt)+'.wav')
            x2 = (x2[:,0] + x2[:,1])/2 # stereo to mono
            x2 = x2/max(x2)*0.5
            f02, sp2,ap2 = pw.wav2world(x2, fs, frame_period =  frame_period)    # use default options
            f0 =np.concatenate((f0,f02[1:-1]), axis=0)
            sp =np.concatenate((sp,sp2[1:-1,:]), axis=0)
            ap = np.concatenate((ap,ap2[1:-1,:]), axis=0)
            
#             f0_note =np.concatenate((f0,f02[1:-1]), axis=0)
#             sp =np.concatenate((sp,sp2[1:-1,:]), axis=0)

            file_cnt +=1
        valid_idx = np.where(f0!=0)
        f0 =  f0[valid_idx]
        sp =  sp[valid_idx]
        ap =  ap[valid_idx]
        idx = get_f0_index_tbl(f0,min_note,max_note,note_step)  
        sp =  sp[idx,:]
            
        ap = ap[idx]
        lin_note = np.linspace(min_note, max_note,num=len(idx))
#         lin_f0 = 440*np.power(2,(lin_note-49)/12)
#         _y = pw.synthesize(lin_f0, sp, ap, fs, frame_period)
#         f02, sp2, ap2 = pw.wav2world(_y, fs, frame_period =  frame_period)
#         print(12*np.log2(f02[1]/440)+49)
        
        key_label = {"sp" :sp,"ap" :ap, "note" :lin_note}
        key_label_dic[v] = key_label
    all_data.append(key_label_dic)


#             f0_note = 12*np.log2(f0/440)+49
#             plt.plot(f0)
#             plt.show()

#         sorted_note = np.array(sorted(i for i in f0_note if i < note_upper+1))
#         del_note = abs(sorted_note[1:] - sorted_note[:-1])
outfile = open(r'train_data/d.pkl', 'wb')
pickle.dump(all_data, outfile)
print('file writes to train_data/d.pkl')
outfile.close()          
            

26 39
gliss_dataset\Abun\a
gliss_dataset\Abun\a2.wav
gliss_dataset\Abun\i
gliss_dataset\Abun\u
gliss_dataset\Abun\e
gliss_dataset\Abun\o
26 38
gliss_dataset\boton\a
gliss_dataset\boton\i
gliss_dataset\boton\u
gliss_dataset\boton\e
gliss_dataset\boton\o
31 41
gliss_dataset\cos\a
gliss_dataset\cos\i
gliss_dataset\cos\u
gliss_dataset\cos\e
gliss_dataset\cos\o
32.0 47.0
gliss_dataset\LISA\a
gliss_dataset\LISA\i
gliss_dataset\LISA\u
gliss_dataset\LISA\e
gliss_dataset\LISA\o
40 52
gliss_dataset\乙彤\a
gliss_dataset\乙彤\i
gliss_dataset\乙彤\u
gliss_dataset\乙彤\e
gliss_dataset\乙彤\o
24 37
gliss_dataset\偉晉\a
gliss_dataset\偉晉\i
gliss_dataset\偉晉\u
gliss_dataset\偉晉\e
gliss_dataset\偉晉\o
26 36
gliss_dataset\君豪\a
gliss_dataset\君豪\a2.wav
gliss_dataset\君豪\i
gliss_dataset\君豪\i2.wav
gliss_dataset\君豪\u
gliss_dataset\君豪\u2.wav
gliss_dataset\君豪\e
gliss_dataset\君豪\e2.wav
gliss_dataset\君豪\o
27.0 39.0
gliss_dataset\士峰\a
gliss_dataset\士峰\i
gliss_dataset\士峰\u
gliss_dataset\士峰\e
gliss_dataset\士峰\o
35 51
gliss_dataset\妤瑄

In [19]:
N = 100
def note2f0(n):
    return 440.0*np.power(2,(n-49)/12.0)

which = all_data[21]['a']
n = np.where(which['note'] == 35.5)[0]

sp = which['sp'][n,:]*np.ones((N,1)) 
f0 = np.zeros((N,))

f0[:] = note2f0(which['note'][n])
ap = which['ap'][n,:]*np.ones((N,1)) 

y = pw.synthesize(f0,sp,ap, fs, frame_period =  frame_period)

sf.write('zan_a.wav', y, fs)


print('done')

done


In [12]:
len(all_data)

23

In [144]:
print(all_data[0]['f0'])

[[3.67630541e-04 3.67765260e-04 3.68199341e-04 ... 3.03416174e-12
  3.03356299e-12 3.03335575e-12]
 [7.21955118e-01 7.27038243e-01 7.39160177e-01 ... 2.47102588e-10
  2.47184292e-10 2.47266410e-10]
 [1.68060795e+00 1.69669017e+00 1.73689298e+00 ... 2.30595534e-10
  2.29999099e-10 2.29782188e-10]
 ...
 [4.68460549e+00 4.69353304e+00 4.71885431e+00 ... 5.87362730e-09
  5.89037604e-09 5.89536631e-09]
 [4.65167749e+00 4.66174276e+00 4.69055038e+00 ... 2.14214553e-08
  2.13710111e-08 2.13558182e-08]
 [6.43532706e-01 6.44139405e-01 6.45967557e-01 ... 8.92702925e-09
  8.89206450e-09 8.87913843e-09]]


In [ ]:
from termcolor import colored
import sys
import numpy as np
import os.path
import soundfile as sf
# import librosa
import pyworld as pw

#28 38 9 男
#28 38 9 女

frame_period = 10.0 # millisecond

aiueo = ['a','i','u','e','o']
for i in range(len(aiueo)):
    wavname = 'data/'+aiueo[i]+'.wav'
    if not os.path.isfile(wavname):
        print(colored('錯誤:找不到音檔','red'),wavname)
        continue

    x, fs = sf.read(wavname)
    x = (x[:,0] + x[:,1])/2 # stereo to mono

    num_of_frames = int(np.floor(len(x)/samples_per_frame))    
    f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options


#   pitch judgement
    f0_note = 12*np.log2(f0[1:-1]/440)+49
    min_his = min(np.histogram(f0_note, bins=13)[0])

In [132]:
x, fs = sf.read('a.wav')

try:
    x = (x[:,0] + x[:,1])/2
    x = x/max(x)*0.3
except:
    pass
frame_period = 10
f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options
f0 = f0[1:]
sp = sp[1:]
ap = ap[1:]

In [133]:
#return f0 index array with len (max_note-min_note)/note_step+1

def get_f0_index_tbl(f0,min_note,max_note,note_step=0.25):
    f0_note = 12*np.log2(f0/440)+49
    out_len = int((max_note-min_note)/note_step+1)
    note_lin =np.linspace(min_note, max_note, num=out_len)
    idx_out = np.zeros(out_len,dtype = int)
    for i in range(out_len):
        idx_out[i] = np.argmin(abs(f0_note-note_lin[i]))
    return idx_out
    

tbl = get_f0_index_tbl(f0,28,38)
    
        
    